In [ ]:
import pickle
import os
import numpy as np
# split train data into actual train and validation set. 
DIRLINK = "aclImdb"
def random_shuffle(threads, labels):
	"""
	this function randomly shuffles the data and labels
	"""
	threads = np.array(threads)
	labels = np.array(labels)
	s = np.arange(labels.shape[0])
	np.random.shuffle(s)
	threads = threads[s]
	labels = labels[s]
	return threads, labels

def get_test_train(traintest):
    positive = [open(DIRLINK+"/"+traintest+"/pos/"+file).read() for file in os.listdir(DIRLINK+"/"+traintest+"/pos/")]
    negative = [open(DIRLINK+"/"+traintest+"/neg/"+file).read() for file in os.listdir(DIRLINK+"/"+traintest+"/neg/")]
    labels = [1]* len(positive)
    data = positive
    data.extend(negative)
    labels.extend([0]*len(negative))
    # shuffle the dataset
    return data, labels 

#train_data, train_labels = get_test_train("train")
#test_data, test_labels = get_test_train("test")   

In [ ]:
"""
# Split train data into actual train and validation sets
LEN_TRAIN = len(train_data)
LEN_TEST = len(test_data)
train_datadf = train_data[:int(round(LEN_TRAIN*0.8))]
train_labeldf = train_labels[:int(round(LEN_TRAIN*0.8))]
val_data = train_data[int(round(LEN_TRAIN*0.8)):]
val_labels = train_labels[int(round(LEN_TRAIN*0.8)):]

print ("Train dataset size is {}".format(len(train_datadf)))
print ("Val dataset size is {}".format(len(val_data)))
print ("Test dataset size is {}".format(len(test_data)))
"""

In [ ]:
"""
# Random sample from train dataset
import random
print (train_data[random.randint(0, len(train_data) - 1)])
#!pip install spacy
#!python -m spacy download en_core_web_sm
"""

In [ ]:
# Functions to do with Tokenization
import tqdm
from tqdm import tqdm_notebook
import spacy
import string
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation
!pip install nltk

def lower_case_remove_punc(parsed):
    return [str(token).lower() for token in parsed if (str(token) not in punctuations)]

def get_ngram(sample, n_gram):
    sample = sample.split(' ')
    output = []
    for i in range(len(sample)-n_gram+1):
        output.append(sample[i:i+n_gram])
    final = [' '.join(x) for x in output]
    return final

def strip_html(sample): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sample)
    return cleantext 

def tokenize_dataset(dataset, name, n_gram):
    token_dataset = {i:[] for i in range(1, n_gram + 1)}
    all_datasets = []
    # we are keeping track of all tokens in dataset
    # in order to create vocabulary later
    for sample in tqdm_notebook(tokenizer.pipe(dataset, disable=['parser', 'tagger', 'ner'], batch_size=512, n_threads=1)):
        sample = str(sample)
        #sample = strip_html(sample)
        for i in range(1, n_gram+1):
            sample_after = get_ngram(sample, i)
            tokens = lower_case_remove_punc(sample_after)
            token_dataset[i].append(tokens)
    for i in range(1, n_gram+1):
        pickle.dump(token_dataset[i],open(name+"tokens_dataset"+str(i)+".pkl", "wb") ) 
        all_datasets.append(token_dataset[i])
    return all_datasets

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(dataset):
    new_dataset = []
    for review in dataset:
        new_review = []
        for word in review:
            new_review.append(wordnet_lemmatizer.lemmatize(word))
        new_dataset.append("".join(new_review))
    return new_dataset

#tokenized_train = tokenize_dataset(lemmatize(train_datadf), "trainlemma", 2)
#tokenized_val = tokenize_dataset(lemmatize(val_data) , "vallemma", 2)
#tokenized_test = tokenize_dataset(lemmatize(test_data), "testlemma", 2)



In [ ]:
"""
unigram_train = pickle.load(open("trainlemmatokens_dataset2.pkl", "rb"))
unigram_val = pickle.load(open("vallemmatokens_dataset2.pkl", "rb"))
unigram_test = pickle.load(open("testlemmatokens_dataset2.pkl", "rb"))
train_labeldf = pickle.load(open("traintokens_labels.pkl", "rb") )
val_labels = pickle.load(open("valtokens_labels.pkl", "rb") )
test_labels = pickle.load(open("testtokens_labels.pkl", "rb") )

pickle.dump(train_labeldf,open("traintokens_labels.pkl", "wb") ) 
pickle.dump(val_labels,open("valtokens_labels.pkl", "wb") ) 
pickle.dump(test_labels,open("testtokens_labels.pkl", "wb") ) 
# these are the labels
"""

In [ ]:
%%javascript
//I'm doing this to be able to move cells up or down easily. 
IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-k', {
    help : 'move up selected cells',
    help_index : 'jupyter-notebook:move-selection-up',
    handler : function (event) {
        IPython.notebook.move_selection_up();
        return false;
    }}
);

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-j', {
    help : 'move down selected cells',
    help_index : 'jupyter-notebook:move-selection-down',
    handler :  function (event) {
        IPython.notebook.move_selection_down();
        return false;
    }}
);

In [ ]:
from collections import Counter
# find the 1000 most common words and subwords. 
max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens, size):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

# flatten the toeknized train to become list of all the tokens
#unigram_flattened_traintokens = [j for sub in unigram_train for j in sub]

#zoken2id, id2token = build_vocab(all_train_tokens)

In [ ]:
#token2id, id2token = build_vocab(unigram_flattened_traintokens)

In [ ]:
# convert token to id in the dataset
def token2index_dataset(tokens_data, token2id):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data
"""
train_data_indices = token2index_dataset(unigram_train)
val_data_indices = token2index_dataset(unigram_val)
test_data_indices = token2index_dataset(unigram_test)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))
"""

In [ ]:
"""
print(len(train_labeldf))
print(len(val_labels))
print(len(test_labels))
"""

In [ ]:
MAX_SENTENCE_LENGTH = 200
# change this based on whicehver n-gram file you're reading in. 

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

class MovieGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def moviegroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

"""
BATCH_SIZE = 32
train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

val_dataset = MovieGroupDataset(val_data_indices, val_labels)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

test_dataset = MovieGroupDataset(test_data_indices, test_labels)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=False)
"""

In [ ]:
# First import torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,2)
    
    def forward(self, data, length):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        # view basically reshapes it, so this averages it out. 
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out

emb_dim = 100
#model = BagOfWords(len(id2token), emb_dim)

In [ ]:
learning_rate = 0.01
num_epochs = 3 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def test_model_routine(train_loader, val_loader, model, criterion, optimizer, num_epochs, learning_rate, scheduler=None):
    acc_per_step_val = []
    acc_per_step_train = []
    for epoch in range(num_epochs):
        acc = []
        for i, (data, lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, length_batch, label_batch = data, lengths, labels
            optimizer.zero_grad()
            outputs = model(data_batch, length_batch)
            loss = criterion(outputs, label_batch)
            loss.backward(retain_graph=True)
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 100 == 0:
                # validate
                val_acc = test_model(val_loader, model) 
                train_acc = test_model(train_loader, model)
                acc.append(val_acc)
                acc_per_step_val.append(val_acc)
                acc_per_step_train.append(train_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}, Train Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc, train_acc))
        #scheduler.step(loss)
        print("Average accuracy is"+ str(np.mean(acc)))
    print("total average accuarcies validation")
    print(acc_per_step_val)
    print("total accuracies train")
    print(acc_per_step_train)
    return acc_per_step_val, acc_per_step_train, model
#test_model_routine(train_loader, val_loader, model, criterion, optimizer, num_epochs, learning_rate)       

In [ ]:
## NOW, LET'S USE ONLY UNIGRAM
unigram_train = pickle.load(open("traintokens_dataset1.pkl", "rb"))
unigram_val = pickle.load(open("valtokens_dataset1.pkl", "rb"))
unigram_test = pickle.load(open("testtokens_dataset1.pkl", "rb"))
train_labeldf = pickle.load(open("traintokens_labels.pkl", "rb") )
val_labels = pickle.load(open("valtokens_labels.pkl", "rb") )
test_labels = pickle.load(open("testtokens_labels.pkl", "rb") )

In [ ]:
len(train_data_indices)

In [ ]:
sizes = [20000, 50000, 100000, 200000]
for size in sizes:
    print("Now varying vocab size" + str(size))
    unigram_flattened_traintokens = [j for sub in unigram_train for j in sub]
    token2id, id2token = build_vocab(unigram_flattened_traintokens, size)
    new_model = BagOfWords(len(id2token), emb_dim)
    optimizer = torch.optim.Adam(new_model.parameters(), lr=learning_rate)
    train_data_indices = token2index_dataset(unigram_train, token2id)
    val_data_indices = token2index_dataset(unigram_val, token2id)
    test_data_indices = token2index_dataset(unigram_test,token2id)
    BATCH_SIZE = 32
    train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=True)
    val_dataset = MovieGroupDataset(val_data_indices, val_labels)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=True)

    test_dataset = MovieGroupDataset(test_data_indices, test_labels)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=False)
    test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate)


In [ ]:
# and now varying embedding size 

sizes = [200, 500, 1000, 5000]
for size in sizes:
    print("Now varying vocab size" + str(size))
    token2id, id2token = build_vocab(unigram_flattened_traintokens, 10000)
    new_model = BagOfWords(len(id2token), size)
    optimizer = torch.optim.Adam(new_model.parameters(), lr=learning_rate)
    train_data_indices = token2index_dataset(unigram_train, token2id)
    val_data_indices = token2index_dataset(unigram_val, token2id)
    test_data_indices = token2index_dataset(unigram_test,token2id)
    BATCH_SIZE = 32
    train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=True)
    val_dataset = MovieGroupDataset(val_data_indices, val_labels)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=True)

    test_dataset = MovieGroupDataset(test_data_indices, test_labels)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=False)
    test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate)

    

In [ ]:
# trying with new optimizer 
token2id, id2token = build_vocab(unigram_flattened_traintokens, 10000)
new_model = BagOfWords(len(id2token), 100)
optimizer = torch.optim.SGD(new_model.parameters(), lr = 0.1, momentum =0.05)
train_data_indices = token2index_dataset(unigram_train, token2id)
val_data_indices = token2index_dataset(unigram_val, token2id)
test_data_indices = token2index_dataset(unigram_test,token2id)
BATCH_SIZE = 32
train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)
val_dataset = MovieGroupDataset(val_data_indices, val_labels)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

test_dataset = MovieGroupDataset(test_data_indices, test_labels)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=False)
test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate)


In [ ]:
# optimizing based on learning rate 

for learning_rate in [0.02, 0.05, 0.1, 0.2, 0.5, 1, 2]:
    print("learning rate is")
    print(str(learning_rate))
    token2id, id2token = build_vocab(unigram_flattened_traintokens, 10000)
    new_model = BagOfWords(len(id2token), 100)
    optimizer = torch.optim.Adam(new_model.parameters(), lr = learning_rate)
    train_data_indices = token2index_dataset(unigram_train, token2id)
    val_data_indices = token2index_dataset(unigram_val, token2id)
    test_data_indices = token2index_dataset(unigram_test,token2id)
    BATCH_SIZE = 32
    train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=True)
    val_dataset = MovieGroupDataset(val_data_indices, val_labels)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=True)

    test_dataset = MovieGroupDataset(test_data_indices, test_labels)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                               batch_size=BATCH_SIZE,
                                               collate_fn=moviegroup_collate_func,
                                               shuffle=False)
    test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate)


In [ ]:
# Now, try with linear annealing. 
token2id, id2token = build_vocab(unigram_flattened_traintokens, 10000)
new_model = BagOfWords(len(id2token), 100)
decay=0.01/num_epochs
optimizer = torch.optim.Adam(new_model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
train_data_indices = token2index_dataset(unigram_train, token2id)
val_data_indices = token2index_dataset(unigram_val, token2id)
test_data_indices = token2index_dataset(unigram_test,token2id)
BATCH_SIZE = 32
train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)
val_dataset = MovieGroupDataset(val_data_indices, val_labels)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

test_dataset = MovieGroupDataset(test_data_indices, test_labels)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=False)

test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate, scheduler=scheduler)

In [ ]:

# Try out Adam decay 
token2id, id2token = build_vocab(unigram_flattened_traintokens, 10000)
new_model = BagOfWords(len(id2token), 100)
optimizer = torch.optim.Adam(new_model.parameters(), lr = 0.1, weight_decay=True)
train_data_indices = token2index_dataset(unigram_train, token2id)
val_data_indices = token2index_dataset(unigram_val, token2id)
test_data_indices = token2index_dataset(unigram_test,token2id)
BATCH_SIZE = 32
train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)
val_dataset = MovieGroupDataset(val_data_indices, val_labels)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

test_dataset = MovieGroupDataset(test_data_indices, test_labels)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=False)

test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate, scheduler=scheduler)

| Annealing | Maximum Validation Accuracy |
|---------------|-----------------------------|
| No annealing     | 85.98                      |
| With annealing        | 85.24                    |



In [ ]:
token2id, id2token = build_vocab(uigram_flattened_traintokens, 100000)
new_model = BagOfWords(len(id2token), 100)
optimizer = torch.optim.Adam(new_model.parameters(), lr = 0.1)
train_data_indices = token2index_dataset(unigram_train, token2id)
val_data_indices = token2index_dataset(unigram_val, token2id)
test_data_indices = token2index_dataset(unigram_test,token2id)
BATCH_SIZE = 32
train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)
val_dataset = MovieGroupDataset(val_data_indices, val_labels)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

test_dataset = MovieGroupDataset(test_data_indices, test_labels)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=False)
acc1, acc2, model = test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate)
"""
Testing the model
"""
test_model(test_loader, model)

In [ ]:
test_data_raw, test_labels_raw = get_test_train("test")   
"""
Help function that tests the model's performance on a dataset
@param: loader - data loader for the dataset to test against
"""


In [ ]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=1,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)
curr = 0
three_example_right = []
three_example_wrong = []

for data, lengths, labels in val_loader:
    if len(three_example_right) < 3 or len(three_example_wrong) < 3:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        if predicted != labels:
            print("Index is "+str(curr))
            print("and model got it wrong")
            three_example_wrong.append(curr)
        else:
            print("Index is "+str(curr))
            print("and model got it right") 
            three_example_right.append(curr)
        curr += 1
    else:
        break



In [ ]:
train_data, train_labels = get_test_train("train")
LEN_TRAIN = len(train_data)

val_data = train_data[int(round(LEN_TRAIN*0.8)):]
val_labels = train_labels[int(round(LEN_TRAIN*0.8)):]

In [ ]:
print("model got these three examples wrong \n")
print(val_data[0])
print("\n")
print(val_data[9])
print("\n")
print(val_data[15])
print("\n")
print("model got these three examples right \n")
print(val_data[1])
print("\n")
print(val_data[2])
print("\n")
print(val_data[3])

In [ ]:
# Let's try varying the batch size! 
new_model = BagOfWords(len(id2token), 100)
optimizer = torch.optim.Adam(new_model.parameters(), lr = 0.1)
train_data_indices = token2index_dataset(unigram_train, token2id)
val_data_indices = token2index_dataset(unigram_val, token2id)
test_data_indices = token2index_dataset(unigram_test,token2id)
BATCH_SIZE = 128
train_dataset = MovieGroupDataset(train_data_indices, train_labeldf)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)
val_dataset = MovieGroupDataset(val_data_indices, val_labels)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=True)

test_dataset = MovieGroupDataset(test_data_indices, test_labels)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=moviegroup_collate_func,
                                           shuffle=False)
acc1, acc2, model = test_model_routine(train_loader, val_loader, new_model, criterion, optimizer, num_epochs, learning_rate)
